run the other notebooks. should give two csv files. this one will combine them and clean up the data too.

In [1]:
from bs4 import BeautifulSoup as bs
import ast
import os
import os.path
from datetime import datetime
from enum import Enum
import re
import pandas as pd
import numpy as np


**for merging with mps.csv**

In [2]:
mp_df = pd.read_csv('mps.csv')
mp_df.Party = mp_df.Party.apply(ast.literal_eval)
mp_df.Parliaments = mp_df.Parliaments.apply(ast.literal_eval)
mps = dict(
    zip(mp_df.Name.apply(lambda x: x.replace('.', '').replace(',', '').lower()), # keys
    zip(mp_df.Name, mp_df.Party, mp_df.Parliaments))) # values
mp_names = list(mps.keys())

In [3]:
# for matching honorific+name in report to actual mp data.
# cannot simply remove honorific as the programmer doesn't have an exhaustive list
# of honorifics, and some are quite rare in everyday use (e.g. Inche Rahamat Bin Kenap).
def honorific_name_to_mp_data(honorific_name):
    honorific_name = honorific_name.replace('.','').replace(',','').lower()
    
    # try the easy way first (find and remove honorific)
    honorific_match = re.match(r'(mr|mrs|ms|mdm|dr|prof|assoc prof|assoc prof dr|inche|encik)', honorific_name)
    if honorific_match:
        name = honorific_name[honorific_match.span()[1]+1:]
        if name in mps.keys():
            return mps[name]

        # slightly harder way (rearranging words)
        for mp_name in mp_names:
            if set(mp_name.split(' ')) == set(name.split(' ')):
                print(f'rearranging matched {honorific_name} to {mp_name}')
                return mps[mp_name]
        
    # the hard way (levenshtein)
    min_levenshtein = 99999
    min_ind = -1
    for i in range(len(mp_names)):
        l_dist = levenshtein(mp_names[i], honorific_name)
        if l_dist < min_levenshtein:
            min_levenshtein = l_dist
            min_ind = i
    print(f'levenshtein matched {honorific_name} to {mp_names[min_ind]}')
    return mps[mp_names[min_ind]]


# borrowed from: https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/
# we use levenshtein as it helps to protect against typos too, like the "asked asked" in:
# https://sprs.parl.gov.sg/search/sprs3topic?reportid=oral-answer-2822
def levenshtein(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

we assume that all pqs are prefaced with #. (non sprs) or # (sprs). ignore follow up qns since we are only interested in mapping mps to topics, and the follow up qns will always be from the same mp and on the same topic.

notes regarding minister titles:
* Minister for Culture, Community and Youth is the only minister title with a comma
* no questions were ever directed to minister mentor

In [13]:
cap_word = r'[A-Z][a-z]+'
cap_words = f'({cap_word})( {cap_word})*'
mccy = 'Minister for Culture, Community and Youth'
minister_for_something = f'({cap_words} )?Minister (for|of) {cap_words}( and (the )?{cap_words})?'
something_minister = f'{cap_words} Minister'
minister_regex = re.compile(f'(({mccy})|({minister_for_something})|({something_minister}))')

def pq_para_to_pq(pq_para):
    para = pq_para[0]
    asker_honorific_name, para = para.split(' asked the ', 1)
    minister_match = re.search(minister_regex, para)
    askee = para[:minister_match.span()[1]].replace(' of ', ' for ')
    question = para[minister_match.span()[1]:].strip()
    asker, asker_party, asker_parls = honorific_name_to_mp_data(asker_honorific_name)
    return [asker, asker_party, asker_parls, askee, question, *pq_para[1:]]


In [5]:
pq_paras = pd.read_csv('pq_paras.csv').values.tolist()

In [14]:
for pq_para in pq_paras:
    pqs.append(pq_para_to_pq(pq_para))

rearranging matched mr p govindaswamy to govindaswamy p
levenshtein matched mr s v lingam to ho see beng
levenshtein matched mr s v lingam to ho see beng
levenshtein matched mr s v lingam to ho see beng


In [15]:
pq_df = pd.DataFrame(pqs, columns=['asker_name', 'asker_party', 'asker_parliaments', 'askee', 'question', 'sitting_date', 'parliament_no', 'report_section'])
pq_df

,asker_name,asker_party,asker_parliaments,askee,question,sitting_date,parliament_no,report_section
0,Ng Kah Ting,[People's Action Party],"(1, 2, 3, 4, 5, 6, 7)",Deputy Prime Minister,whether he is aware that certain sections in ...,1968-01-24,1,Oral Answers to Questions
1,Teong Eng Siong,[People's Action Party],"(1, 2, 3, 4)",Minister for Law and National Development,when work will commence on the construction of...,1968-01-24,1,Oral Answers to Questions
2,Ho See Beng,[People's Action Party],"(0, 1, 2, 3, 4, 5)",Minister for Labour,whether he is aware that the Industrial A...,1968-01-24,1,Oral Answers to Questions
3,Ng Kah Ting,[People's Action Party],"(1, 2, 3, 4, 5, 6, 7)",Deputy Prime Minister,whether he is aware that certain sections in ...,1968-01-24,1,Oral Answers to Questions
4,Teong Eng Siong,[People's Action Party],"(1, 2, 3, 4)",Minister for Law and National Development,when work will commence on the construction of...,1968-01-24,1,Oral Answers to Questions
5,Ho See Beng,[People's Action Party],"(0, 1, 2, 3, 4, 5)",Minister for Labour,whether he is aware that the Industrial A...,1968-01-24,1,Oral Answers to Questions
6,Ng Kah Ting,[People's Action Party],"(1, 2, 3, 4, 5, 6, 7)",Deputy Prime Minister,whether he is aware that certain sections in ...,1968-01-24,1,Oral Answers to Questions
7,Teong Eng Siong,[People's Action Party],"(1, 2, 3, 4)",Minister for Law and National Development,when work will commence on the construction of...,1968-01-24,1,Oral Answers to Questions
8,Ho See Beng,[People's Action Party],"(0, 1, 2, 3, 4, 5)",Minister for Labour,whether he is aware that the Industrial A...,1968-01-24,1,Oral Answers to Questions
9,Ho Kah Leong,[People's Action Party],"(0, 1, 2, 3, 4, 5, 6, 7, 8)",Minister for Defence,", since the conversion of Nicoll Highway into ...",1968-01-24,1,Oral Answers to Questions


In [ ]:
pq_df.to_csv('pqs.csv', index=False)